In [ ]:
import os
import json
import time
import boto3
import pandas as pd
import requests
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [ ]:

bucket_name = "election-2023"


states = {'1': 'ABIA',
 '2': 'ADAMAWA',
 '3': 'AKWA IBOM',
 '4': 'ANAMBRA',
 '5': 'BAUCHI',
 '6': 'BAYELSA',
 '7': 'BENUE',
 '8': 'BORNO',
 '9': 'CROSS RIVER',
 '10': 'DELTA',
 '11': 'EBONYI',
 '12': 'EDO',
 '13': 'EKITI',
 '14': 'ENUGU',
 '15': 'FCT',
 '16': 'GOMBE',
 '17': 'IMO',
 '18': 'JIGAWA',
 '19': 'KADUNA',
 '20': 'KANO',
 '21': 'KATSINA',
 '22': 'KEBBI',
 '23': 'KOGI',
 '24': 'KWARA',
 '25': 'LAGOS',
 '26': 'NASARAWA',
 '27': 'NIGER',
 '28': 'OGUN',
 '29': 'ONDO',
 '30': 'OSUN',
 '31': 'OYO',
 '32': 'PLATEAU',
 '33': 'RIVERS',
 '34': 'SOKOTO',
 '35': 'TARABA',
 '36': 'YOBE',
 '37': 'ZAMFARA'}

In [ ]:
df = pd.read_csv('OG-PLAT.csv')
failed = []
s3 = boto3.client('s3')

In [ ]:
def file_src(html):
    """
    Returns:
    - the link to the # print(all_pus_code)
    
    """
    soup = bs(html, 'lxml')
    urls = soup.iframe['src']
    return urls


In [ ]:

def pus_code(link):

    driver = webdriver.Chrome()
    
    driver.get(link)
    wait = WebDriverWait(driver, 100)
    
    #gets the div element of each polling unit
    element = wait.until(EC.text_to_be_present_in_element((By.TAG_NAME, "body"), "PU Code"))
    html_ward = driver.find_element(By.XPATH, '/html/body/app-root/div/app-activated/div/div/div/div/app-election-lga/div/div/div/div/div[2]/div/div[2]/div[2]/div').get_attribute('innerHTML')
    div = bs(html_ward, 'lxml').find('body')
    pus = div.findAll("div", recursive=False)
    driver.close()
    
    return [i.text for i in pus]

In [ ]:
def save_file(idx,link,pu,state,lga,ward):
    global failed
    try:
        pu = ' '.join(pu.split()[:-2]).split(':')[-1].replace('/', '-')
        
        driver = webdriver.Chrome()
      
        driver.get(link)
        
        wait = WebDriverWait(driver, 100)
        element = wait.until(EC.text_to_be_present_in_element((By.TAG_NAME, "body"), "PU Code"))
        try:
            driver.find_element(By.XPATH,f'/html/body/app-root/div/app-activated/div/div/div/div/app-election-lga/div/div/div/div/div[2]/div/div[2]/div[2]/div/div[{idx+1}]/div/div[2]/button').click()
        except:
            time.sleep(25)
            driver.find_element(By.XPATH,f'/html/body/app-root/div/app-activated/div/div/div/div/app-election-lga/div/div/div/div/div[2]/div/div[2]/div[2]/div/div[{idx+1}]/div/div[2]/button').click()
        
        wait = WebDriverWait(driver, 100)
        element = wait.until(EC.text_to_be_present_in_element((By.TAG_NAME, "body"), "Lg"))
        
        urls = file_src(driver.page_source)
     

        storedAs = urls.rsplit('/')[-1]

        ext = storedAs.rsplit('.')[-1]
        filepath = f"{state}/{lga}/{ward}/{pu}.{ext}"
        try:

            s3.upload_file(storedAs, bucket_name, filepath)
        except:
            download_file(urls,storedAs)
            time.sleep(5)
            s3.upload_file(storedAs, bucket_name, filepath)
       
    except Exception as e:
        failed.append([idx,link])


In [ ]:

cur_dir = os.getcwd()

In [ ]:
def download_file(file_url,filename):
    
    response = requests.get(file_url)
    
    with open(filename, "wb")as f:
        f.write(response.content)

In [ ]:
def main(cur_dir,i):
    os.chdir(cur_dir)
    state = states[i[0].split('=')[-1]]
    lga = i[1].replace('/','-')
    ward = i[2].replace('/','-')
    link = i[3]
    num = i[-1]
    

    _dir = os.path.join(cur_dir, state,lga,ward)
    
    if not os.path.exists(_dir):
        os.makedirs(_dir)
    os.chdir(_dir)
    all_pus_code = pus_code(link)
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    
        future_to_url = {executor.submit(save_file, number,link,all_pus_code[number],state,lga,ward): number for number in range(num)}
        
        for future in concurrent.futures.as_completed(future_to_url):
            pass
    

In [ ]:
with concurrent.futures.ProcessPoolExecutor(max_workers=1) as executor:
    
        future_to_url = {executor.submit(main, cur_dir,i): i for i in df.values[::-1]}
        
        for future in concurrent.futures.as_completed(future_to_url):
            pass
    


In [ ]:
i